In [33]:
from dwave.system import EmbeddingComposite, DWaveSampler
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [3]:
# load the qpu and the auto embedding solver
qpu = DWaveSampler(region='eu-central-1')
print(qpu.solver.name)
sampler = EmbeddingComposite(qpu)

Advantage_system5.4


# Problem setup

We want to find out the statistical behaviour of dwave's qpu.
To do this, we first need to define the problem.

We consider the following two qubit Ising problem:
$$J_{0,1} = -1$$
Such a problem has 2 possible solutions:
$\ket{00}$ and $\ket{11}$.

We need to find out the distribution of the qubits after a certain number of steps.
Statistically we expect them to be equally distributed regardless of :
- The qubits on which they are operated
- The annealing time
- The number of samples per trial
- The mode of the annealing schedule ( simple forward or reverse+forward annealing ) 

# Simple examples

## Forward annealing example

In [4]:
J = {(0, 1): -1}

In [52]:
sampleset = sampler.sample_ising({},J, num_reads=450, annealing_time=2000, auto_scale=False).to_pandas_dataframe()

In [55]:
sampleset['state'] = sampleset.apply(lambda row: f"{int((row[0]+1)/2)} {int((row[1]+1)/2)}", axis=1)
px.bar(sampleset, x='state',y='num_occurrences', title='Frequency distribution')

## Reverse annealing example

In [57]:
initial = {0:-1, 1:-1}  # Set the initial state
reverse_schedule = [ [0.0,1.0], [50,0.5], [100,1.0] ] # Set the annealing schedule, first element is % , the second is ratio of mixing hamiltonian and problem hamiltonian
reverse_anneal_params = dict(anneal_schedule=reverse_schedule,
                             initial_state=initial,
                             reinitialize_state=True)  # Reinitialize to the specified initial state for every anneal-readout cycle. Each anneal begins from the state given in the initial_state parameter.
# See https://docs.dwavesys.com/docs/latest/c_solver_parameters.html#reinitialize-state


In [58]:
sampleset = sampler.sample_ising({}, J, num_reads=1000, **reverse_anneal_params).to_pandas_dataframe()

In [59]:
sampleset['state'] = sampleset.apply(lambda row: f"{int((row[0]+1)/2)} {int((row[1]+1)/2)}", axis=1)
px.bar(sampleset, x='state', title='Frequency distribution')

In [60]:
sampleset

,0,1,chain_break_fraction,energy,num_occurrences,state
0,-1,-1,0.0,-1.0,1,0 0
1,-1,-1,0.0,-1.0,1,0 0
2,-1,-1,0.0,-1.0,1,0 0
3,-1,-1,0.0,-1.0,1,0 0
4,-1,-1,0.0,-1.0,1,0 0
...,...,...,...,...,...,...
995,-1,-1,0.0,-1.0,1,0 0
996,-1,-1,0.0,-1.0,1,0 0
997,1,1,0.0,-1.0,1,1 1
998,-1,-1,0.0,-1.0,1,0 0
